# Algoritmos de Escalonamento

## Importações

In [1]:
from operator import attrgetter
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Definição de Classes

In [2]:
class SystemClock:
    """Gerencia o tempo global do sistema."""
    TIME = 0
    TIME_STATUS = False

    @classmethod
    def tick(cls):
        if not cls.TIME_STATUS:
            cls.TIME += 1
        cls.TIME_STATUS = False

In [3]:
class Scheduler:
    """Classe base para os escalonadores EDF e RM."""
    
    def __init__(self, jobs, attribute, reverse):
        self.jobs = self.sort_jobs(jobs, attribute, reverse)
        self.curr_jobs = self.jobs.copy()
        self.attribute = attribute
        self.reverse = reverse

    @staticmethod
    def sort_jobs(jobs, attribute, reverse=True):
        return sorted(jobs, key=attrgetter(attribute), reverse=reverse)

    def check_jobs(self):
        """Verifica quais tarefas precisam ser despertadas."""
        for job in self.jobs:
            if job.wake():
                self.add_job(job)
                job.update_deadline()
        self.curr_jobs = self.sort_jobs(self.curr_jobs, self.attribute, self.reverse)

    def remove_job(self, job):
        self.curr_jobs.remove(job)

    def add_job(self, job):
        self.curr_jobs.append(job)
        self.curr_jobs = self.sort_jobs(self.curr_jobs, self.attribute, self.reverse)

    def update_time_to_deadline(self):
        for job in self.jobs:
            job.update_deadline()

    def run(self):
        """Método base para execução dos escalonadores."""
        while SystemClock.TIME < CYCLES:
            self.check_jobs()
            if self.curr_jobs:
                curr_job = self.curr_jobs[0]
                job_status = curr_job.run()
                if curr_job.finished():
                    self.remove_job(curr_job)
                if not job_status:
                    print("Uma deadline foi perdida!")
                    break
            SystemClock.tick()
            self.update_time_to_deadline()

In [4]:
class EarliestDeadlineFirst(Scheduler):
    """Escalonador EDF (Menor deadline tem maior prioridade)."""
    
    def __init__(self, jobs):
        super().__init__(jobs, 'to_deadline', reverse=False)


class RateMonotonic(Scheduler):
    """Escalonador RM (Menor período tem maior prioridade)."""
    
    def __init__(self, jobs):
        super().__init__(jobs, 'rm_priority', reverse=True)

In [5]:
class Processor:
    """Processador que executa os escalonadores."""
    
    def __init__(self, jobs):
        self.jobs = jobs

    def edf(self):
        EarliestDeadlineFirst(self.jobs).run()

    def rm(self):
        RateMonotonic(self.jobs).run()

In [6]:
class Job:
    """Classe que representa uma tarefa periódica."""

    def __init__(self, duration, period, name, pid, priority=0):
        self.duration = duration
        self.period = period
        self.priority = priority
        self.rm_priority = 1 / period
        self.deadline = period
        self.runtime = 0
        self.ran = []
        self.active = True
        self.name = name
        self.to_deadline = period - SystemClock.TIME
        self.deadlines = []
        self.pid = pid

    def wake(self):
        """Reativa a tarefa periodicamente."""
        if SystemClock.TIME % self.period == 0 and not self.active:
            print(f'T={SystemClock.TIME} Reiniciando {self.name}')
            self.runtime = 0
            self.active = True
            self.deadline = SystemClock.TIME + self.period
            return True
        return False

    def run(self):
        """Executa a tarefa."""
        if not self.finished():
            SystemClock.TIME_STATUS = True
            self.ran.append(SystemClock.TIME)
            print(f'T={SystemClock.TIME} Executando {self.name}, tempo={self.runtime}')
            self.runtime += 1
            SystemClock.TIME += 1
            if self.check_deadline():
                return False
            self.update_deadline()
        return True

    def finished(self):
        """Verifica se a tarefa foi concluída."""
        if self.runtime >= self.duration:
            self.active = False
            print(f'T={SystemClock.TIME-1} Finalizada {self.name}')
            return True
        return False

    def check_deadline(self):
        """Verifica se a tarefa perdeu a deadline."""
        self.deadlines.append(self.deadline)
        if SystemClock.TIME > self.deadline:
            print(f'Atenção! {self.name} perdeu a deadline.')
            return True
        return False

    def update_deadline(self):
        """Atualiza o tempo restante para a deadline."""
        self.to_deadline = self.deadline - SystemClock.TIME

## Configuração do sistema

In [7]:
job1 = Job(3, 20, "T1", 1)  # C = 3, P = 20
job2 = Job(2, 5, "T2", 2)   # C = 2, P = 5
job3 = Job(2, 10, "T3", 3)  # C = 2, P = 10

jobs = [job1, job2, job3]

CYCLES = 20

## Execução do Algoritmo

In [8]:
processor = Processor(jobs)
processor.rm()

T=0 Executando T2, tempo=0
T=1 Executando T2, tempo=1
T=1 Finalizada T2
T=2 Executando T3, tempo=0
T=3 Executando T3, tempo=1
T=3 Finalizada T3
T=4 Executando T1, tempo=0
T=5 Reiniciando T2
T=5 Executando T2, tempo=0
T=6 Executando T2, tempo=1
T=6 Finalizada T2
T=7 Executando T1, tempo=1
T=8 Executando T1, tempo=2
T=8 Finalizada T1
T=10 Reiniciando T2
T=10 Reiniciando T3
T=10 Executando T2, tempo=0
T=11 Executando T2, tempo=1
T=11 Finalizada T2
T=12 Executando T3, tempo=0
T=13 Executando T3, tempo=1
T=13 Finalizada T3
T=15 Reiniciando T2
T=15 Executando T2, tempo=0
T=16 Executando T2, tempo=1
T=16 Finalizada T2


In [9]:
times = []
for job in jobs:
    for value in job.ran:
        times.append({"Job": job.name, "Start": value, "Finish": value + 1, "Resource": "Running", "PID": job.pid})
    for value in job.deadlines:
        times.append({"Job": job.name, "Start": value, "Finish": value + 0.3, "Resource": "Deadline", "PID": job.pid})

df = pd.DataFrame(times)
df["Delta"] = df["Finish"] - df["Start"]
df.sort_values(by="Start", inplace=True)

In [10]:
jobs = df[df["Resource"] == "Running"]

In [11]:
jobs

,Job,Start,Finish,Resource,PID,Delta
6,T2,0,1.0,Running,2,1.0
7,T2,1,2.0,Running,2,1.0
22,T3,2,3.0,Running,3,1.0
23,T3,3,4.0,Running,3,1.0
0,T1,4,5.0,Running,1,1.0
8,T2,5,6.0,Running,2,1.0
9,T2,6,7.0,Running,2,1.0
1,T1,7,8.0,Running,1,1.0
2,T1,8,9.0,Running,1,1.0
10,T2,10,11.0,Running,2,1.0


## Organização do Escalonamento para plot

In [12]:
schedule = []

for _, row in jobs.iterrows():
  schedule.append(
      {
          "Task": row["Job"],
          "Start": row["Start"],
          "Finish": row["Finish"],
      }
  )

In [13]:
schedule

[{'Task': 'T2', 'Start': 0, 'Finish': 1.0},
 {'Task': 'T2', 'Start': 1, 'Finish': 2.0},
 {'Task': 'T3', 'Start': 2, 'Finish': 3.0},
 {'Task': 'T3', 'Start': 3, 'Finish': 4.0},
 {'Task': 'T1', 'Start': 4, 'Finish': 5.0},
 {'Task': 'T2', 'Start': 5, 'Finish': 6.0},
 {'Task': 'T2', 'Start': 6, 'Finish': 7.0},
 {'Task': 'T1', 'Start': 7, 'Finish': 8.0},
 {'Task': 'T1', 'Start': 8, 'Finish': 9.0},
 {'Task': 'T2', 'Start': 10, 'Finish': 11.0},
 {'Task': 'T2', 'Start': 11, 'Finish': 12.0},
 {'Task': 'T3', 'Start': 12, 'Finish': 13.0},
 {'Task': 'T3', 'Start': 13, 'Finish': 14.0},
 {'Task': 'T2', 'Start': 15, 'Finish': 16.0},
 {'Task': 'T2', 'Start': 16, 'Finish': 17.0}]

In [14]:
def group_tasks(data):
    grouped = []
    schedule_sorted = sorted(schedule, key=lambda x: (x['Task'], x['Start']))
    
    for entry in schedule_sorted:
        if not grouped or grouped[-1]['Task'] != entry['Task'] or grouped[-1]['Finish'] < entry['Start']:
            grouped.append(entry)
        else:
            grouped[-1]['Finish'] = entry['Finish']
    
    return grouped

result = group_tasks(schedule)

In [15]:
result

[{'Task': 'T1', 'Start': 4, 'Finish': 5.0},
 {'Task': 'T1', 'Start': 7, 'Finish': 9.0},
 {'Task': 'T2', 'Start': 0, 'Finish': 2.0},
 {'Task': 'T2', 'Start': 5, 'Finish': 7.0},
 {'Task': 'T2', 'Start': 10, 'Finish': 12.0},
 {'Task': 'T2', 'Start': 15, 'Finish': 17.0},
 {'Task': 'T3', 'Start': 2, 'Finish': 4.0},
 {'Task': 'T3', 'Start': 12, 'Finish': 14.0}]